In [12]:
import numpy as np
import pandas as pd

In [13]:
from tools import generate_samples, data_generator

In [14]:
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [40]:
nn = Sequential()
nn.add(Dense(400, input_shape=(400,), activation='relu'))
nn.add(Dense(400, activation='relu'))
nn.add(Dense(400, activation='sigmoid'))
nn.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
nn.fit_generator(data_generator(), steps_per_epoch=10, epochs=10, verbose=2)

Epoch 1/10
 - 5s - loss: 0.5815 - acc: 0.7349
Epoch 2/10
 - 2s - loss: 0.5436 - acc: 0.7739
Epoch 3/10
 - 4s - loss: 0.5398 - acc: 0.7751
Epoch 4/10
 - 3s - loss: 0.5365 - acc: 0.7748
Epoch 5/10
 - 3s - loss: 0.5374 - acc: 0.7720
Epoch 6/10
 - 3s - loss: 0.5279 - acc: 0.7770
Epoch 7/10
 - 3s - loss: 0.5299 - acc: 0.7732
Epoch 8/10
 - 4s - loss: 0.5274 - acc: 0.7724
Epoch 9/10
 - 3s - loss: 0.5201 - acc: 0.7783
Epoch 10/10
 - 4s - loss: 0.5171 - acc: 0.7763


In [6]:
train_df = pd.read_csv("resources/train.csv", index_col=0)
test_df = pd.read_csv("resources/test.csv", index_col=0)
fst_delta = train_df[train_df.delta == 1]
X, Y = fst_delta.iloc[:, 1:401], fst_delta.iloc[:, 401:]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [41]:
nn.fit(X_train, y_train, epochs=10)

Epoch 1/10
7904/7904 [==============================] - 1s 187us/step - loss: 0.3610 - acc: 0.8604
Epoch 2/10
7904/7904 [==============================] - 1s 143us/step - loss: 0.3016 - acc: 0.8571
Epoch 3/10
7904/7904 [==============================] - 1s 143us/step - loss: 0.2778 - acc: 0.8577
Epoch 4/10
7904/7904 [==============================] - 1s 151us/step - loss: 0.2628 - acc: 0.8622
Epoch 5/10
7904/7904 [==============================] - 1s 146us/step - loss: 0.2515 - acc: 0.8676
Epoch 6/10
7904/7904 [==============================] - 1s 148us/step - loss: 0.2421 - acc: 0.8731
Epoch 7/10
7904/7904 [==============================] - 1s 147us/step - loss: 0.2338 - acc: 0.8785
Epoch 8/10
7904/7904 [==============================] - 1s 150us/step - loss: 0.2265 - acc: 0.8836
Epoch 9/10
7904/7904 [==============================] - 1s 156us/step - loss: 0.2200 - acc: 0.8881
Epoch 10/10
7904/7904 [==============================] - 1s 144us/step - loss: 0.2139 - acc: 0.8925


In [42]:
Y_pred = nn.predict(X_test)

In [43]:
from sklearn.metrics import mean_absolute_error

In [49]:
mean_absolute_error(y_test, Y_pred.round(0))

0.14004807692307694

In [15]:
from keras.applications.vgg16 import VGG16
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [102]:
from keras.layers import Conv2D, Flatten, GlobalAveragePooling2D, MaxPooling2D, Dropout, BatchNormalization, Activation, LeakyReLU
from keras.callbacks import EarlyStopping

In [158]:
cnn = Sequential()
cnn.add(Conv2D(64, (3, 3), input_shape=(20, 20, 1)))
cnn.add(BatchNormalization())
cnn.add(LeakyReLU())
cnn.add(MaxPooling2D())
cnn.add(Conv2D(64, (3, 3)))
cnn.add(BatchNormalization())
cnn.add(LeakyReLU())
cnn.add(MaxPooling2D())
cnn.add(Conv2D(64, (2, 2)))
cnn.add(BatchNormalization())
cnn.add(LeakyReLU())
cnn.add(MaxPooling2D())
cnn.add(GlobalAveragePooling2D())
cnn.add(Dense(400, activation='relu'))
cnn.add(Dense(400, activation='sigmoid'))

In [159]:
cnn.compile('adam', 'binary_crossentropy', ['accuracy'])

In [160]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_76 (Conv2D)           (None, 18, 18, 64)        640       
_________________________________________________________________
batch_normalization_41 (Batc (None, 18, 18, 64)        256       
_________________________________________________________________
leaky_re_lu_38 (LeakyReLU)   (None, 18, 18, 64)        0         
_________________________________________________________________
max_pooling2d_70 (MaxPooling (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 7, 7, 64)          36928     
_________________________________________________________________
batch_normalization_42 (Batc (None, 7, 7, 64)          256       
_________________________________________________________________
leaky_re_lu_39 (LeakyReLU)   (None, 7, 7, 64)          0         
__________

In [161]:
es = EarlyStopping(monitor='acc', patience=3, min_delta=0.05)

In [162]:
cnn.fit(X_train.values.reshape(-1, 20, 20, 1), y_train, epochs=10, callbacks=[es], batch_size=32)

Epoch 1/10
7904/7904 [==============================] - 13s 2ms/step - loss: 0.3695 - acc: 0.8580
Epoch 2/10
7904/7904 [==============================] - 11s 1ms/step - loss: 0.3451 - acc: 0.8625
Epoch 3/10
7904/7904 [==============================] - 11s 1ms/step - loss: 0.3364 - acc: 0.8622
Epoch 4/10
7904/7904 [==============================] - 12s 1ms/step - loss: 0.3305 - acc: 0.8619


In [165]:
Y_pred = cnn.predict(X_test.values.reshape(-1, 20, 20, 1))

In [166]:
mean_absolute_error(y_test, Y_pred)

0.1952824328650243

In [194]:
nn2 = Sequential()
nn2.add(Dense(2000, input_shape=(400,), activation='relu'))
nn2.add(Dropout(0.5))
nn2.add(Dense(2000, activation='relu'))
nn2.add(Dropout(0.5))
nn2.add(Dense(400, activation='sigmoid'))
nn2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
nn2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_120 (Dense)            (None, 2000)              802000    
_________________________________________________________________
dropout_58 (Dropout)         (None, 2000)              0         
_________________________________________________________________
dense_121 (Dense)            (None, 2000)              4002000   
_________________________________________________________________
dropout_59 (Dropout)         (None, 2000)              0         
_________________________________________________________________
dense_122 (Dense)            (None, 400)               800400    
Total params: 5,604,400
Trainable params: 5,604,400
Non-trainable params: 0
_________________________________________________________________


In [195]:
es = EarlyStopping(monitor='loss', patience=6, min_delta=0.01)
nn2.fit(X_train, y_train, epochs=40, callbacks=[es], batch_size=32)
mean_absolute_error(y_test, nn2.predict(X_test))

Epoch 1/40
7904/7904 [==============================] - 20s 2ms/step - loss: 0.3480 - acc: 0.8603
Epoch 2/40
7904/7904 [==============================] - 18s 2ms/step - loss: 0.3013 - acc: 0.8590
Epoch 3/40
7904/7904 [==============================] - 15s 2ms/step - loss: 0.2860 - acc: 0.8610
Epoch 4/40
7904/7904 [==============================] - 15s 2ms/step - loss: 0.2776 - acc: 0.8633
Epoch 5/40
7904/7904 [==============================] - 16s 2ms/step - loss: 0.2717 - acc: 0.8652
Epoch 6/40
7904/7904 [==============================] - 17s 2ms/step - loss: 0.2676 - acc: 0.8672
Epoch 7/40
7904/7904 [==============================] - 18s 2ms/step - loss: 0.2649 - acc: 0.8690
Epoch 8/40
7904/7904 [==============================] - 17s 2ms/step - loss: 0.2632 - acc: 0.8700
Epoch 9/40
7904/7904 [==============================] - 18s 2ms/step - loss: 0.2610 - acc: 0.8716
Epoch 10/40
7904/7904 [==============================] - 17s 2ms/step - loss: 0.2606 - acc: 0.8721
Epoch 11/40
7904/79

0.22363667793686243

In [197]:
nn2.fit_generator(data_generator(), steps_per_epoch=10, epochs=10, verbose=2, callbacks=[es])  # continue training on simulated data
mean_absolute_error(y_test, nn2.predict(X_test)) # notice increase in loss function and drop in accuracy
# one of the possible reasons: distribution of cells on the initial state is *not* uniform

Epoch 1/10
 - 4s - loss: 0.4965 - acc: 0.7705
Epoch 2/10
 - 4s - loss: 0.4987 - acc: 0.7692
Epoch 3/10
 - 3s - loss: 0.5066 - acc: 0.7646
Epoch 4/10
 - 4s - loss: 0.5042 - acc: 0.7657
Epoch 5/10
 - 4s - loss: 0.4984 - acc: 0.7685
Epoch 6/10
 - 4s - loss: 0.4996 - acc: 0.7687
Epoch 7/10
 - 3s - loss: 0.4953 - acc: 0.7702


0.23775724139624377